## 信用卡反欺诈检测之基于imbalanced-learn,XGBoost和LightGBM的有监督学习实现  
>1. 数据及项目来源：[Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud)  
>2. 问题类别：**有监督学习的二分类问题**或者是**无监督学习的异常检测问题**    
>3. 有监督学习方案：使用imbalanced-learn中的BalancedRandomForestClassifier,RUSBoostClassifier以及XGBoost和LightGBM四种模型对数据进行分类  
>4. 无监督学习方案：使用Isolation Forest（孤立森林）对数据进行异常检测  

>5. 思路：单一变量原则，逐渐叠加影响因子  
>>1. 首先对未进行特征缩放的数据进行训练和测试，查看结果，文件：without_feature_scaling_without_feature_selection.ipynb  
>>2. 然后对经过特征缩放但未经过特征选择的数据进行训练和测试，查看结果  文件：with_feature_scaling_without_feature_selection.ipynb  
>>>* 有关特征缩放对本项目使用的四种基于决策树的模型的影响，请参阅本notebook末尾的结果分析部分  
>>3. 最后对经过特征缩放和特征选择的数据进行训练和测试，查看结果  文件：with_feature_scaling_with_feature_selection.ipynb



### I. 加载数据并对其进行初步的探索

In [1]:
# 加载数据前处理的通用库numpy和pandas
import numpy as np
import pandas as pd

In [2]:
# 读取数据为Pandas dataframe格式
data_original = pd.read_csv('creditcard.csv')

In [3]:
#数据为Pandas dataframe格式
type(data_original)

pandas.core.frame.DataFrame

In [4]:
#概览数据，显示前10行
data_original.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [5]:
#显示数据规模，各个特征的数据类型；查看各个特征下是否存在缺失值Null
#根据Kaggle上该数据集的描述，以及本条代码的查看结果，该数据的所有特征均为数值类型，并且没有缺失值，因此，不需要进行one-hot编码，也不需要进行缺失值处理
data_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

In [6]:
#统计各个特征下Null的数量，经过查看，我们发现该数据的确不存在缺失值，与info()的结果一致
data_original.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [7]:
#显示整个数据集中缺失值Null的总数
data_original.isnull().sum().sum()

0

In [8]:
#显示各个特征包含的唯一值的数量
data_original.nunique()

Time      124592
V1        275663
V2        275663
V3        275663
V4        275663
V5        275663
V6        275663
V7        275663
V8        275663
V9        275663
V10       275663
V11       275663
V12       275663
V13       275663
V14       275663
V15       275663
V16       275663
V17       275663
V18       275663
V19       275663
V20       275663
V21       275663
V22       275663
V23       275663
V24       275663
V25       275663
V26       275663
V27       275663
V28       275663
Amount     32767
Class          2
dtype: int64

In [9]:
#显示数据集中各个类别的数量（标签的数量），我们发现两个类别的分布极不平衡
data_original['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [10]:
#显示数据集中各个类别的百分比（标签的百分比），再次验证这是一个类别分布极不平衡的数据集
data_original['Class'].value_counts(normalize = True)*100

0    99.827251
1     0.172749
Name: Class, dtype: float64

In [11]:
#显示各个特征的一些统计学特征
data_original.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


### 数据探索总结:  
1. 数据中一共包含284807个样本  
2. 数据一共包含30个特征列以及一个标签列  
3. 数据的特征列和标签列下均没有缺失值，不需要进行缺失值处理  
4. 数据的30个特征均为连续的数值特征(continuous numerical features)，没有类别特征(categorical features),不需要进行one-hot编码  
5. 数据中两个类别（正常及欺诈）分布极不平衡，正常数据（非欺诈数据）所占比例为99.83%，欺诈数据所占比例为0.17%,因此这是一个非均衡数据集(imbalanced data)的分类问题

### II. 数据集特征列和标签列的分离，训练集和测试集的分割  
>为了尽可能避免数据信息泄露的问题，在对数据进行任何前处理之前，一定要先对数据进行训练集和测试集的分割  
>[参考1：Normalize data before or after split of training and testing data?](https://stackoverflow.com/questions/49444262/normalize-data-before-or-after-split-of-training-and-testing-data)  
>[参考2：Onehotencoding before or after split of training and testing data?](https://stackoverflow.com/questions/55525195/do-i-have-to-do-one-hot-encoding-separately-for-train-and-test-dataset)  
>[参考3：Imputation before or after train test spliting](https://stats.stackexchange.com/questions/95083/imputation-before-or-after-splitting-into-train-and-test)

In [12]:
#首先对数据集进行特征，标签的分离
X = data_original.iloc[:,0:-1]
y = data_original['Class']

In [13]:
X.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [14]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Class, dtype: int64

In [15]:
#然后进行训练集和测试集的分割,75%训练集，25%测试集
#注意，这里我们要采用分层抽样(stratified sampling)的方法，以保证训练集和测试集中类别的比例和总体数据类别的比例基本一致
#此外，如果我们提前获知数据中的某一个特征是关键特征，那么在进行分层抽样的时候，也可以该特征的比例作为参考，进行抽样
#参考链接：https://medium.com/@411.codebrain/train-test-split-vs-stratifiedshufflesplit-374c3dbdcc36
#参考链接：https://zhuanlan.zhihu.com/p/49991313
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42) 

In [16]:
# 显示训练集中各个类别的数量，用于计算scale_pos_weight
y_train.value_counts()

0    213236
1       369
Name: Class, dtype: int64

In [17]:
# scale_pos_weight = sum(negative instances) / sum(positive instances)
# 这个参数是XGBoost和LightGBM两个模型在对非均衡数据进行分类时用于控制类别平衡的最关键的参数
# 实际应用时，也可以考虑使用按上述公式计算得到的值的平方根
# 参考1：https://xgboost.readthedocs.io/en/latest/parameter.html
# 参考2：https://stats.stackexchange.com/questions/243207/what-is-the-proper-usage-of-scale-pos-weight-in-xgboost-for-imbalanced-datasets
scale_pos_weight_1 = 213236 / 369
scale_pos_weight_2 = np.sqrt(scale_pos_weight_1)

In [18]:
scale_pos_weight_1

577.8753387533875

In [19]:
scale_pos_weight_2

24.039037808393818

### III. 对于非均衡数据集（imbalanced data）的处理  
>对于非均衡数据的处理，有多种思路:  
>* **重采样**，包括上采样（Oversampling,也叫过采样）和下采样（Undersampling,也叫欠采样），其基本思路就是将数据中两类的数量调整均衡一些，  
让少的变多,让多的变少,从而使非均衡数据变得均衡.在重采样以后，再利用各种机器学习分类模型对数据进行分类。  
>>- 两种重采样各自的实现方式均有很多种，实践中，我们利用[imbalanced-learn](https://imbalanced-learn.readthedocs.io/en/stable/api.html#module-imblearn.under_sampling)库来完成各种重采样的实现  
>* **利用imbalaced-learn中的分类器**, 这些分类器具有处理非均衡数据的内在机制,比如[BalancedRandomForestClassifier](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.ensemble.BalancedRandomForestClassifier.html#)和[RUSBoostClassifier](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.ensemble.RUSBoostClassifier.html)  
>* **利用XGBoost和LightGBM**,这两种基于GBDT的强分类器，均可以设置参数'scale_pos_weight'来处理这种非均衡数据  
>>- scale_pos_weight = number of negative samples / number of positive samples  
>>- 对于二分类问题，正例(positive)用1表示，反例(negative)用0表示  
>* 模型评估准则(Metrics)：对于非均衡数据，不能再使用accuracy作为评估准则，可以考虑使用f1_score或者专门针对非均衡问题的评估准则，比如  
[geometric_mean_score](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.metrics.geometric_mean_score.html#imblearn.metrics.geometric_mean_score)  

>* [参考1:Dealing With Class Imbalanced Datasets For Classification](https://towardsdatascience.com/dealing-with-class-imbalanced-datasets-for-classification-2cc6fad99fd9)    
>* [参考2:机器学习之类别不平衡问题 (3) —— 采样方法](https://www.cnblogs.com/massquantity/p/9382710.html)  
>* [参考3:机器学习中的非均衡问题(imbalanced data)和应对方法](https://zhuanlan.zhihu.com/p/38687978)  
>* [参考4:机器学习：如何解决机器学习中数据不平衡问题](https://www.jianshu.com/p/be343414dd24)
    

### IV.特征缩放(Feature Scaling)  
>1. 特征缩放的意义?  
*进行缩放后，多维特征将具有相近的尺度，这将帮助梯度下降算法更快地收敛*  
*[参考1:Feature Scaling with scikit-learn](https://benalexkeen.com/feature-scaling-with-scikit-learn/)*  
*[参考2:为什么 feature scaling 会使 gradient descent 的收敛更好?](https://www.zhihu.com/question/37129350)*  
*[参考3:机器学习（一）- feature scaling](https://blog.csdn.net/mike112223/article/details/74923096)*  
*[参考4:sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) *  
*[参考5:特征缩放的几种方法](https://www.cnblogs.com/HuZihu/p/9761161.html) *  
>1. *仅对连续数值特征(continuous numeric features)进行特征缩放，对离散类别特征(discrete categorical features)不做缩放*  
*[参考1:Should you ever standardise binary variables?](https://stats.stackexchange.com/questions/59392/should-you-ever-standardise-binary-variables)*  
*[参考2:Dummy variables, is it necessary to standardize them?
](https://stackoverflow.com/questions/50557129/dummy-variables-is-necessary-to-standardize-them)*  
>3. *最常用的三种特征缩放方法: 最大最小值归一化(min-max scaling)，标准化(standardization)以及稳健缩放(RobustScaler):*  
   *- min-max scaling(normalization):*  
   $ X_{minmax} = \frac{X - X_{min}}{X_{max} - X_{min}} $  
   *- standardization(Z-score):*  
   $ X_{std} = \frac{X - \mu }{\sigma } $  *($\mu 是均值$, $\sigma$是标准差)*  
   *- robustscaler:*  
   $ X_{rb} = \frac{X - Q_1(x) }{Q_3(x) - Q_1(x)} $ *($Q_1$是第一四分位数， $Q_3$是第三四分位数)*  
   [参考1:Feature Scaling with scikit-learn](https://benalexkeen.com/feature-scaling-with-scikit-learn/) 
>4. *其他的特征缩放的方法: PowerTransformer and QuantileTransformer*
>5. *[其他参考1：Compare the effect of different scalers on data with outliers](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py) *  
*[其他参考2：Which advantages does MinMax scaling have over a standard scaling using the mean and the standard deviation?](https://www.quora.com/Which-advantages-does-MinMax-scaling-have-over-a-standard-scaling-using-the-mean-and-the-standard-deviation) *  
*[其他参考3：How to Identify Outliers in your Data？](https://machinelearningmastery.com/how-to-identify-outliers-in-your-data/) *  
*[其他参考4：How to Make Your Machine Learning Models Robust to Outliers？](https://www.kdnuggets.com/2018/08/make-machine-learning-models-robust-outliers.html) *      
>6. *__注意__: 需要先把数据拆分成训练集与验证集，在训练集上计算出需要的数值（如均值和标准值），对训练集数据做标准化/归一化处理（不要在整个数据集上做标准化/归一化处理，因为这样会将验证集的信息带入到训练集中，这是一个非常容易犯的错误），然后再用之前计算出的数据（如均值和标准值）对验证集数据做相同的标准化/归一化处理.*  
*[参考1：Difference between preprocessing train and test set before and after splitting](https://stats.stackexchange.com/questions/267012/difference-between-preprocessing-train-and-test-set-before-and-after-splitting) *




#### 基于之前对三种特征缩放方法的探索，我们最终选择采用标准化(standardization)  
>详情请参见文件: with_feature_scaling_without_feature_selection.ipynb

In [20]:
# 标准化-Standardization(Z-score)
from sklearn.preprocessing import StandardScaler

scaler_std = StandardScaler()  
X_train_copy_2 = X_train.copy(deep=True)
X_train_std = scaler_std.fit_transform(X_train_copy_2)

In [21]:
type(X_train_std)

numpy.ndarray

In [22]:
type(y_train)

pandas.core.series.Series

In [24]:
y_train_values = y_train.to_numpy()

In [25]:
type(y_train_values)

numpy.ndarray

### V. 特征选择(Feature Selection)  
>* 特征选择的本质：剔除冗余特征或不相关特征，对原始特征进行降维  
>* 特征选择的意义：提高模型预测的准确性，减少过拟合-提升模型的泛化能力，提升模型训练速度，  
>* 特征选择在机器学习流程中的位置：[特征选择要放在特征缩放之后进行](https://stackoverflow.com/questions/46062679/right-order-of-doing-feature-selection-pca-and-normalization)  
>* 特征选择有多种方法，基本可以归为三类：过滤(Filter)，包裹(Wrapper) 和嵌入(Embedded)  
>>* Filter方法不依赖于特定的模型，对特征重要性的判断不需要先经过模型训练，因此该方法是模型独立的，常用的方法有:  
Welch's t-Test, Fisher score, Chi-Squared test, Relief-based, CFS(Correlation-based feature selection),  
FCBF(Fast correlation-based filter)等等。
>>* Wrapper方法需要先指定一个机器学习模型，并进行训练，然后根据模型训练的结果来对特征的重要性进行判断。问题在于，究竟应该选择哪种模型并没有定论，基于不同种类的模型得到的特征重要性结果可能不一致，解决的方法：一是投票，二是[欲训练什么算法，就选择该算法进行评估](https://zhuanlan.zhihu.com/p/74198735)。Wrapper常用的方法有：Recursive Feature Elimination(RFE,递归式特征消除), Simulated Annealing(SA，模拟退火), 以及Genetic Algorithm(GA, 遗传算法)等。其中，[遗传算法最能保证获得全局最优特征子集](http://www.feat.engineering/genetic-algorithms.html),但这种方法运行非常耗时，效率低下，仅建议在运算资源充足，项目时间充足的情况下使用。  
>>* Embedded方法，利用正则化思想，将部分特征属性的权重变成零，从而完成特征选择。 常见的正则化有L1的Lasso，L2的Ridge和混合的Elastic Net。
>* [参考1:Overview of feature selection methods](https://towardsdatascience.com/overview-of-feature-selection-methods-a2d115c7a8f7)  
[参考2:Machine Learning Explained: Regularization](http://enhancedatascience.com/2017/07/04/machine-learning-explained-regularization/)  
[参考3:机器学习（六）：特征选择方法—Filter,Wrapper,Embedded](https://zhuanlan.zhihu.com/p/120924870)  
[参考4:基于 Jupyter 的特征工程手册：特征选择（四）](https://zhuanlan.zhihu.com/p/138758083)  
[参考5：特征选择，经典三刀](https://zhuanlan.zhihu.com/p/24635014)  
[参考6：特征工程笔记](https://www.jianshu.com/p/03284dd5e0bf)  

>* 本项目使用一种[Wrapper方法:Boruta](https://github.com/scikit-learn-contrib/boruta_py)进行特征选择,使用BorutaPy需注意：  
>>* BorutaPy在设计算法时，是基于RandomForest的，而对其他一些基于树模型的高级算法，比如XGBoost和LightGBM的兼容性则不好，实际应用中经常出现各种错误，具体请查阅官方github的相关issues；  
>>* 本人在尝试将XGBoost和LightGBM带入BorutaPy进行特征选择时，也因兼容性问题，报错而无法继续；  
>>* 目前，根据测试，BorutaPy与传统的RandomForest以及经过改良的专门针对非均衡数据的BalancedRandomForest具有良好的兼容性
>* Boruta在进行特征选择的过程中，会试图找到包含对模型预测有用信息的所有特征  
>* [其他有关Boruta的参考1：Select Important Variables using Boruta Algorithm](https://www.datasciencecentral.com/profiles/blogs/select-important-variables-using-boruta-algorithm)  
>* [其他有关Boruta的参考2：Boruta explained exactly how you wished someone explained to you](https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a)  
>* [其他有关Boruta的参考3：Feature selection? You are probably doing it wrong](https://towardsdatascience.com/feature-selection-you-are-probably-doing-it-wrong-985679b41456)  
>* 目前已有基于Boruta的方法，但采用XGBoost模型进行特征选择的库，比如：[BoostARoota](https://github.com/chasedehan/BoostARoota)  


In [66]:
# 加载相关库
from boruta import BorutaPy
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import RUSBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.utils import class_weight
from sklearn.tree import DecisionTreeClassifier

In [27]:
#对于非均衡数据的class_weight的计算方法
#参考1：https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53696
#参考2：https://stackoverflow.com/questions/44716150/how-can-i-assign-a-class-weight-in-keras-in-a-simple-way
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes = np.unique(y_train),y = y_train)
class_weight_dict = dict(enumerate(class_weights))

In [28]:
type(class_weights)

numpy.ndarray

In [29]:
type(class_weight_dict)

dict

In [30]:
class_weights

array([  0.50086524, 289.43766938])

In [32]:
class_weight_dict

{0: 0.5008652385150725, 1: 289.43766937669375}

In [33]:
# 返回数据集的特征名
# 参考链接：https://stackoverflow.com/questions/19482970/get-list-from-pandas-dataframe-column-headers
Columns = X_train.columns.to_numpy()
Columns

array(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9',
       'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18',
       'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27',
       'V28', 'Amount'], dtype=object)

In [41]:
#定义BalancedRandomForestClassifier，使用所有CPU核进行计算
#注意class_weight的计算方法
#注意，BorutaPy作者建议将max_depth设置为3-7
#此处构建模型只需要设定一些基本值即可，不涉及超参数调节的问题
brf = BalancedRandomForestClassifier(n_jobs=-1, class_weight = class_weight_dict, max_depth=7,random_state=42)
feat_selector_brf = BorutaPy(brf, n_estimators='auto', verbose=2, random_state=42)

>**1. 使用BalancedRandomForestClassifier进行特征选择**

In [45]:
#执行特征选择
feat_selector_brf.fit(X_train_std,y_train_values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	30
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	30
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	30
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	30
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	30
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	30
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	30
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	21
Iteration: 	9 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	21
Iteration: 	10 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	21
Iteration: 	11 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	21
Iteration: 	12 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	21
Iteration: 	13 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	21
Iteration: 	14 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	21
Iteration: 	15 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	21
Iteration: 	16 / 100
Confirmed: 	7
Tentative: 	2
Rejected: 	21
I

BorutaPy(alpha=0.05,
         estimator=BalancedRandomForestClassifier(bootstrap=True,
                                                  class_weight={0: 0.5008652385150725,
                                                                1: 289.43766937669375},
                                                  criterion='gini', max_depth=7,
                                                  max_features='auto',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_samples_leaf=2,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=60, n_jobs=-1,
                                                  oob_score=False,
                                                  random_state=<mtrand.R

In [47]:
#Boruta进行特征选择过程后，结果包含三类.第一类，确定对模型预测有价值的特征，即确定要保留的特征；第二类，确定对模型预测无价值的特征，即确定被剔除的特征；第三类，#Boruta无法确定是否对模型预测有价值，即无法确定是否这些特征应该被保留还是剔除，是否保留由我们决定。其中support_中包含着第一类，而support_weak_中包含着第三类。
confirmed_brf = feat_selector_brf.support_

In [49]:
type(confirmed_brf)

numpy.ndarray

In [48]:
confirmed_brf

array([False,  True, False,  True,  True, False, False, False, False,
       False, False,  True,  True, False,  True, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False])

In [50]:
#返回确定要保留的特征名
features_confirmed_brf = Columns[confirmed_brf]
features_confirmed_brf

array(['V1', 'V3', 'V4', 'V11', 'V12', 'V14', 'V24'], dtype=object)

In [52]:
#返回BorutaPy无法决定是否保留或剔除的特征，即存疑特征
tentative_brf = feat_selector_brf.support_weak_

In [53]:
tentative_brf

array([ True, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [54]:
#返回存疑特征名,这些特征是否保留由我们自己决定。
features_tentative_brf = Columns[tentative_brf]
features_tentative_brf

array(['Time', 'V9'], dtype=object)

In [62]:
#值为1的特征最重要，为Boruta确定要保留的，值为2的特征为存疑特征，是否保留由我们自己决定，其他特征为被剔除的特征
rank_brf = feat_selector_brf.ranking_
rank_brf

array([ 2,  1, 14,  1,  1, 11,  9,  9,  4,  2,  3,  1,  1,  6,  1, 17,  7,
       12,  7, 22, 19, 15, 16, 23,  1, 19,  5, 13, 21, 19])

In [63]:
#参考链接：https://www.codenong.com/6618515/
features_brf_sorted = Columns[rank_brf.argsort()]
features_brf_sorted


array(['V14', 'V1', 'V24', 'V3', 'V4', 'V12', 'V11', 'V9', 'Time', 'V10',
       'V8', 'V26', 'V13', 'V16', 'V18', 'V7', 'V6', 'V5', 'V17', 'V27',
       'V2', 'V21', 'V22', 'V15', 'V25', 'Amount', 'V20', 'V28', 'V19',
       'V23'], dtype=object)

In [56]:
#返回确定保留的特征名
features_filtered_brf_narrow = features_confirmed_brf
#返回确定保留的特征加存疑特征名
features_filtered_brf_broad = np.unique(np.concatenate((features_confirmed_brf,features_tentative_brf),0))

In [57]:
features_filtered_brf_narrow

array(['V1', 'V3', 'V4', 'V11', 'V12', 'V14', 'V24'], dtype=object)

In [58]:
features_filtered_brf_broad

array(['Time', 'V1', 'V11', 'V12', 'V14', 'V24', 'V3', 'V4', 'V9'],
      dtype=object)

In [55]:
#features_filtered_narrow只保留了确定特征，而features_filtered_broad保留了确定和存疑特征
X_train_filtered_brf_narrow = feat_selector_brf.transform(X_train_std)
#weak = True,包含存疑特征
X_train_filtered_brf_broad = feat_selector_brf.transform(X_train_std,weak = True)

In [60]:
X_train_filtered_brf_narrow.shape

(213605, 7)

In [61]:
X_train_filtered_brf_broad.shape

(213605, 9)

### VI. 训练模型并使用[Hyperopt](http://hyperopt.github.io/hyperopt/)进行超参数的调优  
>* hyperopt是一种通过**贝叶斯优化(Bayesian Optimization)**来调整参数的工具  
>* 三种调参方法GridSearch,RandomSearch以及Bayesian Search的对比可参见：  
>>* [Intuitive Hyperparameter Optimization : Grid Search, Random Search and Bayesian Search](https://towardsdatascience.com/intuitive-hyperparameter-optimization-grid-search-random-search-and-bayesian-search-2102dbfaf5b)  
>>* [贝叶斯优化: 一种更好的超参数调优方式](https://zhuanlan.zhihu.com/p/29779000)  
>* 本项目采用十折交叉验证法进行参数调优,模型评估准则采用专门针对非均衡数据的准则geometric_mean_score  
>* 思路：  
>>1. 首先对未进行特征缩放的数据进行训练和测试，查看结果  
>>2. 然后对经过特征缩放但未经过特征选择的数据进行训练和测试，查看结果  
>>3. 最后对经过特征缩放和特征选择的数据进行训练和测试，查看结果  
>* [RandomForest调参参考](https://www.cnblogs.com/pinard/p/6160412.html)  


In [87]:
import json
import time
from sklearn.metrics import make_scorer
from imblearn.metrics import geometric_mean_score
from sklearn.utils import class_weight
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import RUSBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score 
from hyperopt import fmin, tpe, atpe, hp, STATUS_OK, Trials, space_eval # (atpe) adaptive TPE 算法是hyperopt最新版本加入的新算法
from sklearn.model_selection import StratifiedKFold

In [88]:
#对于非均衡数据的class_weight的计算方法
#参考1：https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53696
#参考2：https://stackoverflow.com/questions/44716150/how-can-i-assign-a-class-weight-in-keras-in-a-simple-way
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes = np.unique(y_train),y = y_train)
class_weight_dict = dict(enumerate(class_weights))

In [89]:
type(class_weights)

numpy.ndarray

In [90]:
class_weights

array([  0.50086524, 289.43766938])

In [91]:
type(class_weight_dict)

dict

In [92]:
class_weight_dict

{0: 0.5008652385150725, 1: 289.43766937669375}

In [108]:
#BalancedRandomForest分类器的训练
start = time.time()
def brdf(params):
    # ip = params["imputer"]
    # del params["imputer"]
    # sc = params["scaler"]
    # del params["scaler"]
    brdf_clf = BalancedRandomForestClassifier(**params)
    str_kfold = StratifiedKFold(
        n_splits=10, shuffle=True, random_state=42
    )  # 注意随机数random_state保持一致，以便复现结果
    # 参考链接：https://stackoverflow.com/questions/39782243/how-to-use-cross-val-score-with-random-state
    gms = make_scorer(geometric_mean_score)
    metric = cross_val_score(
        brdf_clf,
        # Data_to_opt(sc)[0],
        # Data_to_opt(sc)[1],
        X_train_filtered_brf_broad,
        y_train,
        cv=str_kfold,
        scoring = gms, 
        n_jobs=-1,  
    ).mean()  
    return {"loss": -metric, "status": STATUS_OK}

space4brdf = {
    "n_estimators": hp.choice("n_estimators", range(100, 320, 20)),
    "max_depth": hp.choice("max_depth", range(1, 70)),  #! max_depth 影响模型的复杂程度
    # "max_features": 1,
    "max_features": hp.choice("max_features", range(1, 9)),
    "class_weight": class_weight_dict, # 这是BalancedRandomForestClassifier用于控制各类平衡的关键参数
    "warm_start": True,
    "n_jobs": -1,
    "random_state": 42,  # 注意保持随机状态的一致性，以便复现结果
    #"imputation": hp.choice("imputation", ["dropna", "SI", "MI"]),
    #"scaling_method": hp.choice("scaling_method", ["min_max", "std"]),
}

rstate = np.random.RandomState(42)
trials = Trials()
best = fmin(
    brdf, space4brdf, algo=tpe.suggest, max_evals=30, trials=trials, rstate=rstate
)  #! fmin返回的是这些最佳参数在其列表中的索引，而不是直接返回最佳参数本身
# print(best)
print(space_eval(space4brdf, best))  #! space_eval()输出最佳参数本身而不是索引
# print(lgt(best))
print(trials.best_trial["result"]["loss"])
# print(trials.best_trial["result"])

# 把最终搜索到的最有超参数写入到一个json文件
# 参考链接: https://stackabuse.com/scikit-learn-save-and-restore-models/
with open("brdf.json", "w") as f:
    f.write(json.dumps({"f1": trials.best_trial["result"]["loss"], "Best params": space_eval(space4brdf, best)}))
hyperparams_brdf = space_eval(space4brdf, best)
stop = time.time()
print(f"Training time: {stop - start:.3f}s")

100%|██████████| 30/30 [27:41<00:00, 55.39s/trial, best loss: -0.9403892094531207]
{'class_weight': {0: 0.5008652385150725, 1: 289.43766937669375}, 'max_depth': 60, 'max_features': 1, 'n_estimators': 120, 'n_jobs': -1, 'random_state': 42, 'warm_start': True}
-0.9403892094531207
Training time: 1661.680s


In [109]:
#RUSBoostClassifier的训练及参数调优
start = time.time()
def rusb(params):
    # ip = params["imputer"]
    # del params["imputer"]
    # sc = params["scaler"]
    # del params["scaler"]
    rusb_clf = RUSBoostClassifier(**params)
    str_kfold = StratifiedKFold(
        n_splits=10, shuffle=True, random_state=42
    )  # 注意随机数random_state保持一致，以便复现结果
    # 参考链接：https://stackoverflow.com/questions/39782243/how-to-use-cross-val-score-with-random-state
    gms = make_scorer(geometric_mean_score)
    metric = cross_val_score(
        rusb_clf,
        # Data_to_opt(sc)[0],
        # Data_to_opt(sc)[1],
        X_train_filtered_brf_broad,
        y_train,
        cv=str_kfold,
        scoring = gms, 
        n_jobs=-1,  
    ).mean()  
    return {"loss": -metric, "status": STATUS_OK}

space4rusb = {
    "n_estimators": hp.choice("n_estimators", range(50, 320, 20)),
    # "max_depth": hp.choice("max_depth", range(1, 70)),  
    "learning_rate": hp.uniform("learning_rate", 0, 1),
    # "max_features": 1,
    # "max_features": hp.choice("max_features", range(1, 30)),
    # "class_weight": class_weight_dict,
    # "warm_start": True,
    # "n_jobs": -1,
    "random_state": 42,  
    #"imputation": hp.choice("imputation", ["dropna", "SI", "MI"]),
    #"scaling_method": hp.choice("scaling_method", ["min_max", "std"]),
}

rstate = np.random.RandomState(42)
trials = Trials()
best = fmin(
    rusb, space4rusb, algo=tpe.suggest, max_evals=30, trials=trials, rstate=rstate
)  #! fmin返回的是这些最佳参数在其列表中的索引，而不是直接返回最佳参数本身
# print(best)
print(space_eval(space4rusb, best))  #! space_eval()输出最佳参数本身而不是索引
# print(lgt(best))
print(trials.best_trial["result"]["loss"])
# print(trials.best_trial["result"])

with open("rusb.json", "w") as f:
    f.write(json.dumps({"f1": trials.best_trial["result"]["loss"], "Best params": space_eval(space4rusb, best)}))
hyperparams_rusb = space_eval(space4rusb, best)
stop = time.time()
print(f"Training time: {stop - start:.3f}s")

100%|██████████| 30/30 [36:31<00:00, 73.06s/trial, best loss: -0.9446299227077104]
{'learning_rate': 0.2562966159166658, 'n_estimators': 70, 'random_state': 42}
-0.9446299227077104
Training time: 2191.754s


In [110]:
#XGBoostClassifier的训练和参数调优
start = time.time()
def xgb(params):
    # ip = params["imputer"]
    # del params["imputer"]
    # sc = params["scaler"]
    # del params["scaler"]
    xgb_clf = XGBClassifier(**params)
    str_kfold = StratifiedKFold(
        n_splits=10, shuffle=True, random_state=42
    )  #!Here the random state should be the same as that in the model
    # ?https://stackoverflow.com/questions/39782243/how-to-use-cross-val-score-with-random-state
    gms = make_scorer(geometric_mean_score)
    metric = cross_val_score(
        xgb_clf,
        # Data_to_opt(sc)[0],
        # Data_to_opt(sc)[1],
        X_train_filtered_brf_broad,
        y_train,
        cv=str_kfold,
        scoring = gms, 
        n_jobs=-1,  
    ).mean()  
    return {"loss": -metric, "status": STATUS_OK}

space4xgb = {
    "max_depth": hp.choice("max_depth", range(3, 20)),  
    "learning_rate": hp.uniform("learning_rate", 0, 1),
    "n_estimators": hp.choice("n_estimators", [50, 100, 150, 200, 250, 300]),
    # "objective": "multi:softmax",
    "objective": "binary:logistic",
    "scale_pos_weight": hp.choice("scale_pos_weight", [scale_pos_weight_1,scale_pos_weight_2]),
    "n_jobs": -1,
    "gamma": hp.randint("gamma", 10),
    "min_child_weight": hp.choice("min_child_weight", range(1, 10)),
    "subsample": hp.uniform("subsample", 0.1, 1.0),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.1, 1.0),
    "random_state": 42,  
    "tree_method": "hist",
    # "imputation": hp.choice("imputation", ["dropna", "SI", "MI"]),
    # "scaling_method": hp.choice("scaling_method", ["min_max", "std"]),
}

rstate = np.random.RandomState(42)
trials = Trials()
best = fmin(
    xgb, space4xgb, algo=tpe.suggest, max_evals=30, trials=trials, rstate=rstate
)  #! fmin返回的是这些最佳参数在其列表中的索引，而不是直接返回最佳参数本身
# print(best)
print(space_eval(space4xgb, best))  #! space_eval()输出最佳参数本身而不是索引
# print(lgt(best))
print(trials.best_trial["result"]["loss"])
# print(trials.best_trial["result"])


with open("xgb.json", "w") as f:
    f.write(json.dumps({"f1": trials.best_trial["result"]["loss"], "Best params": space_eval(space4xgb, best)}))
hyperparams_xgb =  space_eval(space4xgb, best)
stop = time.time()
print(f"Training time: {stop - start:.3f}s")

100%|██████████| 30/30 [16:12<00:00, 32.41s/trial, best loss: -0.9318785441163249]
{'colsample_bytree': 0.39779512373111203, 'gamma': 5, 'learning_rate': 0.326749198175752, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 50, 'n_jobs': -1, 'objective': 'binary:logistic', 'random_state': 42, 'scale_pos_weight': 577.8753387533875, 'subsample': 0.625032357077646, 'tree_method': 'hist'}
-0.9318785441163249
Training time: 972.361s


In [111]:
#LightGBMClassifier的训练和恶参数调优
start = time.time()
def lgbm(params):
    # ip = params["imputer"]
    # del params["imputer"]
    # sc = params["scaler"]
    # del params["scaler"]
    lgbm_clf = LGBMClassifier(**params)
    str_kfold = StratifiedKFold(
        n_splits=10, shuffle=True, random_state=42
    )  
    gms = make_scorer(geometric_mean_score)
    metric = cross_val_score(
        lgbm_clf,
        # Data_to_opt(sc)[0],
        # Data_to_opt(sc)[1],
        X_train_filtered_brf_broad,
        y_train,
        cv=str_kfold,
        scoring = gms, 
        n_jobs=-1,  
    ).mean()  
    return {"loss": -metric, "status": STATUS_OK}

space4lgbm = {
    # 参考链接： https://lightgbm.readthedocs.io/en/latest/Parameters.html#max_bin
    # max_bin: int, default =255, >1,
    # smaller max_bin, faster speed, maybe underfitting; larger max_bin, slower speed, maybe overfitting
    "max_bin": 63,
    "num_leaves": hp.choice("num_leaves", range(100, 500)),  # * the larger this value, the more complex the model is
    "max_depth": hp.choice("max_depth", range(3, 32)),
    "learning_rate": hp.uniform("learning_rate", 0.01, 0.2),
    # "n_estimators": hp.choice("n_estimators", [50, 100, 150, 200, 250, 300]), #! n_estimators has bug here
    "num_boost_round": hp.choice("num_boost_round", range(50, 500)),  #! this is an alias of n_estimators but no bug
    # "objective": "multiclass",
    "objective": "binary",
    "n_jobs": -1,
    # "class_weight": "balanced",  #! This set should be done when the classes are imbalanced
    "scale_pos_weight": hp.choice("scale_pos_weight", [scale_pos_weight_1,scale_pos_weight_2]), #控制非均衡是数据中各个类别的平衡
    "min_split_gain": hp.uniform("gamma", 0, 50),  #! this is the 'gamma' in xgboost but its type is float now
    "min_child_weight": hp.uniform("min_child_weight", 0, 10),
    "min_child_samples": hp.randint("min_child_samples", 20),  #! too large may cause underfitting
    "subsample": hp.uniform("subsample", 0.1, 1.0),
    "subsample_freq": hp.choice("subsample_freq", range(1, 30)),  #! k means perform bagging at every k iteration
    "colsample_bytree": hp.uniform("colsample_bytree", 0.1, 1.0),
    "random_state": 42,  #! Here the random state should be the same as that in the stratify Kfold setttings
    "gpu_use_dp": False,  #! for result's reproducibility
    "device": "gpu",
    "gpu_platform_id": 0,  # *OpenCL platform ID   小规模的CPU会快，大规模的GPU会快
    "gpu_device_id": 0,  # *OpenCL device ID
    # "imputation": hp.choice("imputation", ["dropna", "SI", "MI"]),
    # "scaling_method": hp.choice("scaling_method", ["min_max", "std"]),
}#!fmin needs this random state for reproducibility, and all the random seed should be the same as above.
rstate = np.random.RandomState(42)
trials = Trials()
best = fmin(
    lgbm, space4lgbm, algo=tpe.suggest, max_evals=30, trials=trials, rstate=rstate
)  #! fmin返回的是这些最佳参数在其列表中的索引，而不是直接返回最佳参数本身
# print(best)
print(space_eval(space4lgbm, best))  #! space_eval()输出最佳参数本身而不是索引
# print(lgt(best))
print(trials.best_trial["result"]["loss"])
# print(trials.best_trial["result"])


with open("lgbm.json", "w") as f:
    f.write(json.dumps({"f1": trials.best_trial["result"]["loss"], "Best params": space_eval(space4lgbm, best)}))
hyperparams_lgbm = space_eval(space4lgbm, best)
stop = time.time()
print(f"Training time: {stop - start:.3f}s")

100%|██████████| 30/30 [10:05<00:00, 20.19s/trial, best loss: -0.9216389649107372]
{'colsample_bytree': 0.6637068249287105, 'device': 'gpu', 'gpu_device_id': 0, 'gpu_platform_id': 0, 'gpu_use_dp': False, 'learning_rate': 0.023186776303918973, 'max_bin': 63, 'max_depth': 4, 'min_child_samples': 5, 'min_child_weight': 9.995289243479617, 'min_split_gain': 19.079058859531695, 'n_jobs': -1, 'num_boost_round': 125, 'num_leaves': 317, 'objective': 'binary', 'random_state': 42, 'scale_pos_weight': 577.8753387533875, 'subsample': 0.30670170310598655, 'subsample_freq': 29}
-0.9216389649107372
Training time: 606.540s


### VII.测试，检验模型的泛化能力

#### Feature Scaling on Test Set  
> *Fit on training set and transform on test set.*

In [112]:
# std
X_train_copy_5 = X_train.copy(deep=True)
scaler_std.fit(X_train_copy_5)
X_test_copy_2 = X_test.copy(deep=True)
X_test_std = scaler_std.transform(X_test_copy_2)

#### Feature Selection on Test Set  
> *Fit on training set and transform on test set.*

In [113]:
#features_filtered_narrow只保留了确定特征，而features_filtered_broad保留了确定和存疑特征
X_test_filtered_brf_narrow = feat_selector_brf.transform(X_test_std)
#weak = True,包含存疑特征
X_test_filtered_brf_broad = feat_selector_brf.transform(X_test_std,weak = True)

#### Model Refit

In [114]:
# BalancedRandomForestClassifier Refit
brdf_refit = BalancedRandomForestClassifier(**hyperparams_brdf)
brdf_refit.fit(X_train_filtered_brf_broad, y_train)

BalancedRandomForestClassifier(bootstrap=True,
                               class_weight={0: 0.5008652385150725,
                                             1: 289.43766937669375},
                               criterion='gini', max_depth=60, max_features=1,
                               max_leaf_nodes=None, min_impurity_decrease=0.0,
                               min_samples_leaf=2, min_samples_split=2,
                               min_weight_fraction_leaf=0.0, n_estimators=120,
                               n_jobs=-1, oob_score=False, random_state=42,
                               replacement=False, sampling_strategy='auto',
                               verbose=0, warm_start=True)

In [115]:
# RUSBoostClassifier Refit
rusb_refit = RUSBoostClassifier(**hyperparams_rusb)
rusb_refit.fit(X_train_filtered_brf_broad, y_train)

RUSBoostClassifier(algorithm='SAMME.R', base_estimator=None,
                   learning_rate=0.2562966159166658, n_estimators=70,
                   random_state=42, replacement=False,
                   sampling_strategy='auto')

In [116]:
# XGBoostClassifier Refit
xgb_refit = XGBClassifier(**hyperparams_xgb)
xgb_refit.fit(X_train_filtered_brf_broad, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.39779512373111203, gamma=5,
              learning_rate=0.326749198175752, max_delta_step=0, max_depth=3,
              min_child_weight=2, missing=None, n_estimators=50, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=577.8753387533875,
              seed=None, silent=None, subsample=0.625032357077646,
              tree_method='hist', verbosity=1)

In [117]:
# LGBMClassifier Refit
lgbm_refit = LGBMClassifier(**hyperparams_lgbm)
lgbm_refit.fit(X_train_filtered_brf_broad, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.6637068249287105, device='gpu',
               gpu_device_id=0, gpu_platform_id=0, gpu_use_dp=False,
               importance_type='split', learning_rate=0.023186776303918973,
               max_bin=63, max_depth=4, min_child_samples=5,
               min_child_weight=9.995289243479617,
               min_split_gain=19.079058859531695, n_estimators=100, n_jobs=-1,
               num_boost_round=125, num_leaves=317, objective='binary',
               random_state=42, reg_alpha=0.0, reg_lambda=0.0,
               scale_pos_weight=577.8753387533875, silent=True,
               subsample=0.30670170310598655, subsample_for_bin=200000,
               subsample_freq=29)

#### Model Test

In [118]:
# BalancedRandomForestClassifier Test
y_test_pred = brdf_refit.predict(X_test_filtered_brf_broad)
gms = geometric_mean_score(y_test, y_test_pred, average="binary")  
print("Final geometric_mean_score:", gms)

Final geometric_mean_score: 0.9336766629451787


In [119]:
# RUSBoostClassifier Test
y_test_pred = rusb_refit.predict(X_test_filtered_brf_broad)
gms = geometric_mean_score(y_test, y_test_pred, average="binary")  
print("Final geometric_mean_score:", gms)

Final geometric_mean_score: 0.9355062114318377


In [120]:
# XGBoostClassifier Test
y_test_pred = xgb_refit.predict(X_test_filtered_brf_broad)
gms = geometric_mean_score(y_test, y_test_pred, average="binary")  
print("Final geometric_mean_score:", gms)

Final geometric_mean_score: 0.9411245957300625


In [121]:
# LGBMClassifier Test
y_test_pred = lgbm_refit.predict(X_test_filtered_brf_broad)
gms = geometric_mean_score(y_test, y_test_pred, average="binary")  
print("Final geometric_mean_score:", gms)

Final geometric_mean_score: 0.9202913971283998


### VIII.结果汇总：  
![运行结果](./results_std_fs.png)  
![运行结果](./barchart.png)  
![运行结果](./barchart_2.png)

### IX.结果分析  
>1. 经特征选择，四种模型的训练速度均得到了一定程度的提升  
>2. 对于LightGBM,特征选择对其预测能力略有降低  
>3. 对于其他三种模型，特征选择其预测能力均有一定程度的提升  
>4. BorutaPy分析得到的两个存疑特征：'Time' 和 'V9', 对于除LightGBM之外的其他三种模型均是关键特征  
>5. 本项目中，最终对信用卡数据进行欺诈检测表现最优的组合是：  
>>* 模型：XGBoostClassifier  
>>* 特征缩放：standardization（标准化）  
>>* 特征：\['Time', 'V1', 'V11', 'V12', 'V14', 'V24', 'V3', 'V4', 'V9'\]  
>>* 最终得分(geometric_mean_score): 0.9411  
>>* 训练时间(秒)：972.361s